In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

In [2]:
from deepface import DeepFace
import os
import pandas as pd

2023-09-26 12:45:37.927930: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-26 12:45:37.930123: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 12:45:37.977365: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 12:45:37.978077: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 12:45:38.792136: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [4]:
img_dir = "/var/wd_smit/internal_repos/backups/images"
models = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "Dlib", "ArcFace"]
embeddings = []
face_coords = []
names = []

df = pd.DataFrame({"embedding": [], "facial_area": []})

for img_name in os.listdir(img_dir):
    img_path = os.path.join(img_dir, img_name)
    embedding = DeepFace.represent(img_path, model_name=models[1])
    embeddings.append(embedding[0]["embedding"])
    face_coords.append(embedding[0]["facial_area"])
    names.append(img_name)

    dftemp = pd.DataFrame(embedding)
    df = pd.concat([df, dftemp], ignore_index=True)

df["name"] = names

df['embedding'] = df['embedding'].apply(lambda x: ','.join(map(str, x)))
# df['facial_area'] = df['facial_area'].apply(lambda x: ','.join(map(str, x)))

In [ ]:
len(df['embedding'][2])

In [ ]:
df['facial_area']

In [5]:
for i in range(len(df['facial_area'])):
    df['facial_area'][i] = list(df['facial_area'][i])

In [8]:
# Create an index
es.indices.create(index='base_index', body={
  "mappings": {
    "properties": {
      "embedding": {
        "type": "dense_vector",
        "dims": 128
      },
      "facial_area": {
        "type": "dense_vector",
        "dims": 4
      },
      "name":{
        "type": "keyword"
      }
    }
  }
})

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'base_index'})

In [9]:
for i, row in df.iterrows():
    # Convert the row to a dictionary
    doc = row.to_dict()

    # Index the document
    es.index(index='base_index', body=doc)

BadRequestError: BadRequestError(400, 'mapper_parsing_exception', 'Failed to parse object: expecting token of type [VALUE_NUMBER] but found [FIELD_NAME]')

In [ ]:
queryvec = df['embedding'][2]

In [ ]:
res = es.search(index='my-index', body={
    'query': {
        'match': {
            'embedding': queryvec
        }
    }
})

In [ ]:
res

In [ ]:
res['hits']

In [ ]:
resp = es.search(index="base_index", query={"match_all": {}})

In [ ]:
resp

In [7]:
es.indices.delete(index='*')

ObjectApiResponse({'acknowledged': True})